<a href="https://colab.research.google.com/github/prathik-l/Prathik-Lathish-Babu/blob/main/Autonomous_Driving_Dataset_semantic_segmentation_predicting_future_traffic_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
import torchvision
import torchvision.transforms as T
from collections import defaultdict, deque
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import ast
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
import torchvision.transforms as transforms
import cv2
import os,sys,matplotlib,re
from PIL import Image
from skimage import exposure
import matplotlib.pyplot as plt
import matplotlib.image as immg
from torchvision.io import read_image
import matplotlib
from torchvision.utils import draw_bounding_boxes
import math

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

In [ ]:
bbox_path ='https://drive.google.com/file/d/1RINkFagfbrLE_vKYQlkQFChgpW1uogsK/view?usp=share_link'

image_folder_path ='https://drive.google.com/drive/folders/1fF3LJBtLJ-4fohY4LRq5gKqexHlbPfkV?usp=share_link'


In [ ]:
data_bbox = pd.read_csv(bbox_path)

data_bbox

,"<!DOCTYPE html><html><head><meta name=""google"" content=""notranslate""><meta http-equiv=""X-UA-Compatible"" content=""IE=edge;""><style nonce=""GRuQZRLZGu90bTy68ph0Gg"">@font-face{font-family:'Roboto';font-style:italic;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xIIzc.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:300;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmSU5fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu4mxP.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:500;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmEU9fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:700;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmWUlfBBc9.ttf)format('truetype');}</style><meta name=""referrer"" content=""origin""><title>images_w_boxes.csv - Google Drive</title><meta property=""og:title"" content=""images_w_boxes.csv""><meta property=""og:type"" content=""article""><meta property=""og:site_name"" content=""Google Docs""><meta property=""og:url"" content=""https://drive.google.com/file/d/1RINkFagfbrLE_vKYQlkQFChgpW1uogsK/view?usp=share_link&amp;usp=embed_facebook""><link rel=""shortcut icon"" href=""https://ssl.gstatic.com/images/branding/product/1x/drive_2020q4_32dp.png""><link rel=""stylesheet"" href=""https://fonts.googleapis.com/css?family=Google+Sans:300",400,500,"700"" nonce=""GRuQZRLZGu90bTy68ph0Gg""><link rel=""stylesheet"" href=""https://www.gstatic.com/_/apps-fileview/_/ss/k=apps-fileview.v.z3L6k94ms5s.L.X.O/am=AEA/d=0/rs=AO0039uOTw8Ymwpo5l9PTWjc6VVsmrpNTg"" nonce=""GRuQZRLZGu90bTy68ph0Gg""><script nonce=""F6kmWHzf3xOWigtzB8k13g"">_docs_flag_initialData={""docs-ails"":""docs_cold""","docs-fwds:""docs_sdf""","docs-crs:""docs_crs_nfd""",docs-fl:1,docs-l1lc:2,"docs-l1lm:""CHS""",docs-l2lc:2,...,0.83,0.84,0.85,0.86,1].11,[[null.2,null.203,null.204,"https://www.gstatic.com/og/_/js/k=og.qtm.en_US.7CmTUC1dl7E.es5.O/rt=j/m=qabr,q_dnp,qapid/exm=qaaw,qadd,qaid,qein,qhaw,qhba,qhbr,qhch,qhga,qhid,qhin,qhpr/d=1/ed=1/rs=AA2YrTtDCsxgvNhkmCP_swCt8NfowK4K5Q]]]]",};this.gbar_=this.gbar_||{};(function(_){var window=this;
0,try{,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Copyright The Closure Library Authors.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SPDX-License-Identifier: Apache-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,*/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,var Wj=function(a,b,c){_.ie.log(46,{att:a,max:b,url:c})},Yj=function(a,b,c){_.ie.log(47,{att:a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,}catch(e){_._DumpException(e)},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,})(this.gbar_);,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,// Google Inc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
data_bbox.info()
# getting info about the 

NameError: ignored

In [8]:
data_bbox = ['bbox_x1'][0]

In [18]:
i=6

img = read_image(image_folder_path + data_bbox['img_path'][i]())

bbox = [data_bbox['bbox_x1'][i],data_bbox['bbox_y1'][i],data_bbox['bbox_x2'][i],data_bbox['bbox_y2'][i]]

bbox = torch.tensor(bbox, dtype=torch.int)

bbox = bbox.unsqueeze(0)

print(bbox)

img=torchvision.utils.draw_bounding_boxes(img, bbox, width=3, colors=(255,255,0))
img = torchvision.transforms.ToPILImage()(img)
display(img)

NameError: ignored

In [17]:
df = data_bbox.copy()

AttributeError: ignored

In [19]:
df

NameError: ignored

In [ ]:
len(df['img_path'].unique().tolist())

NameError: ignored

# Create data class:

In [ ]:
class CarDataset(object):
    def __init__(self, df, IMG_DIR, transforms=None):
        self.a = 0
        self.df = df
        self.img_dir = IMG_DIR
        self.image_ids = self.df['img_path'].unique().tolist()
        self.transforms = transforms
        
    def __len__(self):
        return len(self.image_ids)
        
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        records = self.df[self.df['img_path'] == image_id]
        image = cv2.imread(self.img_dir+image_id,cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        boxes = records[['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2']].to_numpy()
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        labels = torch.ones((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([idx])
        target['area'] = torch.as_tensor(area, dtype=torch.float32)
        target['iscrowd'] = torch.zeros((records.shape[0],), dtype=torch.int64)
    
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
        return image.clone().detach(), target, image_id

In [ ]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
def get_train_transform():
    return A.Compose([
        A.Flip(0.5),
        ToTensorV2()
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        ToTensorV2()
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

Let`s show how work our class dataset

In [ ]:
CD = CarDataset(df, image_folder_path, get_train_transform())

NameError: ignored

In [2]:
fig = plt.figure(figsize=(14, 10))
sample = CD[1]
# print(sample[1])
img_int = torch.tensor(sample[0] * 255, dtype=torch.uint8)
plt.imshow(draw_bounding_boxes(
    img_int, sample[1]['boxes'], width=4
).permute(1, 2, 0))

NameError: ignored

# Split data and create dataloaders:

In [3]:
image_ids = df['img_path'].unique()
valid_ids = image_ids[-665:]
train_ids = image_ids[:-665]
valid_df = df[df['img_path'].isin(valid_ids)]
train_df = df[df['img_path'].isin(train_ids)]
train_df.shape,valid_df.shape

NameError: ignored

In [4]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = CarDataset(train_df, image_folder_path, get_train_transform())
valid_dataset = CarDataset(valid_df, image_folder_path, get_valid_transform())

indices = torch.randperm(len(train_dataset)).tolist()
train_data_loader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn
)


valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn
)

NameError: ignored

# Model:
I use fasterrcnn_resnet50_fpn pre-trained on COCO dataset

In [ ]:
num_classes = 2  # 1 class (car) + background

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [ ]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Train our model:

In [ ]:
def train_one_epoch(model, optimizer, loader, device, epoch):
    model.to(device)
    model.train()
    
    all_losses = []
    all_losses_dict = []
    
    for images, targets,_ in tqdm(loader):
        images = list(image.to(device) for image in images)
        targets = [{k: torch.tensor(v).to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets) 
        losses = sum(loss for loss in loss_dict.values())
        loss_dict_append = {k: v.item() for k, v in loss_dict.items()}
        loss_value = losses.item()
        
        all_losses.append(loss_value)
        all_losses_dict.append(loss_dict_append)
        
        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping trainig") 
            print(loss_dict)
            sys.exit(1)
        
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        
    all_losses_dict = pd.DataFrame(all_losses_dict)
    print("Epoch {}, lr: {:.6f}, loss: {:.6f}, loss_classifier: {:.6f}, loss_box: {:.6f}, loss_rpn_box: {:.6f}, loss_object: {:.6f}".format(
        epoch, optimizer.param_groups[0]['lr'], np.mean(all_losses),
        all_losses_dict['loss_classifier'].mean(),
        all_losses_dict['loss_box_reg'].mean(),
        all_losses_dict['loss_rpn_box_reg'].mean(),
        all_losses_dict['loss_objectness'].mean()
    ))

In [ ]:
num_epochs=3

for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, train_data_loader, device, epoch)

NameError: ignored

# Show resault:

In [ ]:
def apply_nms(orig_prediction, iou_thresh=0.2):
    

    keep = torchvision.ops.nms(orig_prediction['boxes'], orig_prediction['scores'], iou_thresh)
    
    final_prediction = orig_prediction
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]
    
    return final_prediction

In [ ]:
img,target,_ = valid_dataset[43]
model.eval()
with torch.no_grad():
    prediction = model([img.to('cuda')])[0]
    
nms_prediction = apply_nms(prediction, iou_thresh=0.7)

NameError: ignored

In [ ]:
print("REAL BOXES")
fig = plt.figure(figsize=(14, 10))
sample = valid_dataset[43]
img_int = torch.tensor(sample[0] * 255, dtype=torch.uint8)
plt.imshow(draw_bounding_boxes(
    img_int, sample[1]['boxes'], width=4
).permute(1, 2, 0))

REAL BOXES


NameError: ignored

<Figure size 1008x720 with 0 Axes>

In [ ]:
print("PREDICTED BOXES")
fig = plt.figure(figsize=(14, 10))
sample = valid_dataset[43]
img_int = torch.tensor(sample[0] * 255, dtype=torch.uint8)
plt.imshow(draw_bounding_boxes(img_int,
    nms_prediction['boxes'][nms_prediction['scores'] > 0.8], width=4
).permute(1, 2, 0))

PREDICTED BOXES


NameError: ignored

<Figure size 1008x720 with 0 Axes>